# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [ ]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [ ]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [ ]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## List all downloaded mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show RFC1213 object tree

In [ ]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [ ]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## Agent code

In [ ]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import datetime
import socket
import os

def get_snmp_type(var) -> str:
    if type(var) == str:
        return "string"
    elif type(var) == int:
        return "integer"
    else:
        return "<unknown>"

def main():

    model_name = "Tesla Model S 2024"
    manufacturing_date = datetime.datetime(year= 2024, month= 2, day= 29, hour= 14, min= 30, second= 58).strftime("%Y-%m-%d %H:%M:%S")
    ac_temp = 23
    battery_percentage = 86
    horsepower = 670
    max_speed = 267
    interior_lights = 2

    door_index1 = 1
    door_index2 = 2
    door_index3 = 3
    door_index4 = 4

    door_status1 = 1
    door_status2 = 2
    door_status3 = 2
    door_status4 = 1

    door_window1 = 3
    door_window2 = 2
    door_window3 = 1
    door_window4 = 3

    objects = { ".1.3.6.1.3.1234.1.1.0" : model_name, 
                ".1.3.6.1.3.1234.1.2.0" : manufacturing_date,
                ".1.3.6.1.3.1234.1.3.0" : ac_temp,
                ".1.3.6.1.3.1234.1.4.0" : battery_percentage,
                ".1.3.6.1.3.1234.1.5.0" : horsepower,
                ".1.3.6.1.3.1234.1.6.0" : max_speed,
                ".1.3.6.1.3.1234.1.7.0" : interior_lights,
                ".1.3.6.1.3.1234.1.8.1.1.1" : door_index1,
                ".1.3.6.1.3.1234.1.8.1.1.2" : door_index2,
                ".1.3.6.1.3.1234.1.8.1.1.3" : door_index3,
                ".1.3.6.1.3.1234.1.8.1.1.4" : door_index4,

                ".1.3.6.1.3.1234.1.8.1.2.1" : door_status1,
                ".1.3.6.1.3.1234.1.8.1.2.2" : door_status2,
                ".1.3.6.1.3.1234.1.8.1.2.3" : door_status3,
                ".1.3.6.1.3.1234.1.8.1.2.4" : door_status4,

                ".1.3.6.1.3.1234.1.8.1.3.1" : door_window1,
                ".1.3.6.1.3.1234.1.8.1.3.2" : door_window2,
                ".1.3.6.1.3.1234.1.8.1.3.3" : door_window3,
                ".1.3.6.1.3.1234.1.8.1.3.4" : door_window4 }
    

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        print("Usage: agent.py <request-type> <MIB-oid> [type] [<new-value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid in objects:
            print(oid)
            print(get_snmp_type(objects[oid]))
            print(objects[oid])
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid in objects and len(sys.argv) == 5:
            content_type = sys.argv[3]
            new_content = sys.argv[4]
            if content_type == "i"
                new_content = int(new_content)
            elif content_type == "s"
                new_content = str(new_content)
            print(oid)
            print(get_snmp_type(new_content))
            print(new_content)
            objects[oid] = new_content
        else:
            print("NONE")
    elif request_type == "-gn" #GETNEXT request
        
        if oid in objects: #caso de pegar o valor da proxima chave
            #pegar o próximo objeto a partir da entrada na tabela
            
        else: #caso de pegar o próximo nó folha, achar a primeira chave que contém o oid
            #iterar sobre as chaves checando qual chave contem o oid e retornar a primeira
            
        print("NONE")

if __name__ == "__main__":
    main()

## Testing

In [ ]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0

In [ ]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.3.0 s teste

In [ ]:
%%writefile MYMIB.txt

MIB-T2 DEFINITIONS ::= BEGIN

IMPORTS
    OBJECT-GROUP FROM SNMPv2-CONF
    OBJECT-TYPE, experimental FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

teslaMIB MODULE-IDENTITY
    LAST-UPDATED "202406290000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO "admin@localhost"
    DESCRIPTION "MIB for acessing information about tesla cars."
::= { experimental 1234 }

t_Info OBJECT IDENTIFIER ::= { teslaMIB 1 }

t_ModelName OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "Tesla model name."
    ::= { t_Info 1 }

t_ManDate OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "Tesla manufacturing date"
    ::= { t_Info 2 }

t_ACTemp OBJECT-TYPE
    SYNTAX  INTEGER
    ACCESS  read-write
    STATUS  mandatory
    DESCRIPTION
            "Temperature set on the air conditioner on celsius."
    ::= { t_Info 3 }

t_BatteryPercentage OBJECT-TYPE
    SYNTAX  INTEGER (0..100)
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "Tesla battery percentage"
    ::= { t_Info 4 }

t_HP OBJECT-TYPE
    SYNTAX  INTEGER
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "Tesla HP"
    ::= { t_Info 5 }

t_MaxSpeed OBJECT-TYPE
    SYNTAX  INTEGER
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "Tesla max speed reached"
    ::= { t_Info 6 }

t_InterLigths OBJECT-TYPE
    SYNTAX  INTEGER {
                    on(1),
                    off(2),
                }
    ACCESS  read-only
    STATUS  read-write
    DESCRIPTION
            "Tesla interior lights config"
    ::= { t_Info 7 }

t_DoorTable OBJECT-TYPE 
    SYNTAX SEQUENCE OF t_DoorEntry
    ACCESS not-accessible
    STATUS mandatory
    DESCRIPTION
    "A list of door info entries." 
    ::= { t_Info 8 }

t_DoorEntry OBJECT-TYPE
    SYNTAX  t_DoorEntry
    ACCESS  not-accessible
    STATUS  mandatory
    DESCRIPTION "Information entry about vehicle doors."
    INDEX   { t_DoorIndex }
    ::= { t_DoorTable 1 }

t_DoorEntry ::=
    SEQUENCE {
        t_DoorStatus    INTEGER,
        t_DoorWindow    INTEGER
    }

t_DoorIndex OBJECT-TYPE
    SYNTAX  INTEGER { front-right(1), front-left(2), rear-right(3), rear-left(4) }
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION "Door identifier."
    ::= { t_DoorEntry 1 }

t_DoorStatus OBJECT-TYPE
    SYNTAX  INTEGER { open(1), closed(2) }
    ACCESS  read-write
    STATUS  mandatory
    DESCRIPTION "Door status, tracks opening and closing of doors."
    ::= { t_DoorEntry 2 }

t_DoorWindow OBJECT-TYPE
    SYNTAX  INTEGER { open(1), half-open(2), closed(3) }
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION "Door window status."
    ::= { t_DoorEntry 3 }

END

In [ ]:
%%bash

snmpget -v2c -c public -M +. -m +MYMIB localhost myCurrentDateTime.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myHostname.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myFileContent.0


In [ ]:
%%bash

snmpset -v2c -c private -M +. -m +MYMIB localhost myFileContent.0 s teste